# Base Analysis

### Setup

#### Load Libraries

In [1]:
library(tidyverse)
library(lubridate)
library(sf)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.2
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

Linking to GEOS 3.6.1, GDAL 2.1.3, PROJ 4.9.3


#### Parameters

In [5]:
## First save csv files into a folder called data within your working directory

census_path <- "data/ACS_15_5YR_B02001.csv.zip"
nd_path <- "data/nd_statewide_2019_08_13.csv.zip"
wa_path <- "data/wa_statewide_2019_08_13.csv"
county_shapefiles_path <- "data/tl_2016_us_county/tl_2016_us_county.shp"
#wa_path_2 <- "wa_statewide_2019_08_13.csv"

nd_state_fip <- 38

#### Load Data

In [6]:
census_data <-
    census_path %>%
    read_csv(skip = 1) %>%
    select(
        geography = Geography, 
        total_pop = `Estimate; Total:`, 
        total_na = `Estimate; Total: - American Indian and Alaska Native alone`
    ) %>%
    separate(geography, c("county", "state"), sep = ", ", remove = FALSE) %>%
    mutate(
        na_pop_prop = (total_na/total_pop) * 100,
        state = state %>% str_to_lower(),
        county = county %>% str_to_lower()
    )

county_shapefiles_data <-
    county_shapefiles_path %>%
    st_read() %>%
    rename(county = "NAMELSAD")

nd_data <-
    nd_path %>%
    read_csv() %>%
    rename(county = "county_name") %>%
    mutate(
        state = "north dakota",
        county = county %>% str_to_lower()
    ) %>%
    left_join(census_data, by = c("state", "county"))

#wa_data <-
#    wa_path %>%
#    read_csv()

#wa_data_2 <-
    #wa_path_2 %>%
    #read_csv()

Multiple files in zip: reading 'ACS_15_5YR_B02001.csv'
Parsed with column specification:
cols(
  .default = col_double(),
  Id = col_character(),
  Id2 = col_character(),
  Geography = col_character()
)
See spec(...) for full column specifications.


Reading layer `tl_2016_us_county' from data source `/Users/michaelspencer/stanford_coding/biglocalpolicing/data/tl_2016_us_county/tl_2016_us_county.shp' using driver `ESRI Shapefile'
Simple feature collection with 3233 features and 17 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: -179.2311 ymin: -14.60181 xmax: 179.8597 ymax: 71.44106
epsg (SRID):    4269
proj4string:    +proj=longlat +datum=NAD83 +no_defs


Multiple files in zip: reading 'nd_statewide_2019_08_13.csv'
Parsed with column specification:
cols(
  raw_row_number = col_character(),
  date = col_date(format = ""),
  time = col_time(format = ""),
  location = col_character(),
  county_name = col_character(),
  subject_age = col_double(),
  subject_race = col_character(),
  subject_sex = col_character(),
  type = col_character(),
  violation = col_character(),
  outcome = col_character(),
  raw_Race = col_character()
)


### Shapefiles Exploration

In [7]:
county_shapefiles_data %>% head()

STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,county,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<MULTIPOLYGON [°]>
31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,NA,NA,NA,A,1477895811,10447360,+41.9158651,-096.7885168,MULTIPOLYGON (((-97.01952 4...
53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,NA,NA,NA,A,680956787,61588406,+46.2946377,-123.4244583,MULTIPOLYGON (((-123.4364 4...
35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,NA,NA,NA,A,6016761713,29147306,+34.3592729,-104.3686961,MULTIPOLYGON (((-104.5674 3...
31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,NA,A,2169240199,22877180,+40.7835474,-096.6886584,MULTIPOLYGON (((-96.9106 40...
31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,NA,NA,NA,A,1489645187,1718484,+40.1764918,-098.0468422,MULTIPOLYGON (((-98.27367 4...
72,085,01804523,72085,Las Piedras,Las Piedras Municipio,13,H1,G4020,490,41980,NA,A,87748363,32509,+18.1871483,-065.8711890,MULTIPOLYGON (((-65.91048 1...


### Census Data Exploration

In [11]:
## Proportions are shown as percentages (ie 4.9 = 4.9%) to avoid scientific notation

census_data %>%
    filter(geography %>% str_detect("North Dakota"))

geography,county,state,total_pop,total_na,na_pop_prop
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
"Adams County, North Dakota",adams county,north dakota,2341,63,2.6911576
"Barnes County, North Dakota",barnes county,north dakota,11097,104,0.9371902
"Benson County, North Dakota",benson county,north dakota,6794,3735,54.9749779
"Billings County, North Dakota",billings county,north dakota,969,5,0.5159959
"Bottineau County, North Dakota",bottineau county,north dakota,6634,164,2.4721134
"Bowman County, North Dakota",bowman county,north dakota,3221,38,1.1797578
"Burke County, North Dakota",burke county,north dakota,2208,26,1.1775362
"Burleigh County, North Dakota",burleigh county,north dakota,88223,3345,3.7915283
"Cass County, North Dakota",cass county,north dakota,162500,1898,1.1680000


### North Dakota Analysis

In [12]:
nd_data %>%
    head()

raw_row_number,date,time,location,county,subject_age,subject_race,subject_sex,type,violation,outcome,raw_Race,state,geography,total_pop,total_na,na_pop_prop
<chr>,<date>,<drtn>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
91444,2011-05-04,09:58:00,"94, 291",barnes county,101,white,male,vehicular,390902: Exceeded speed limit,citation,White,north dakota,"Barnes County, North Dakota",11097,104,0.9371902
138130,2011-12-28,11:41:00,gateway at 55th st,grand forks county,101,white,male,vehicular,391022: Failed to yield at intersection,citation,White,north dakota,"Grand Forks County, North Dakota",68979,1789,2.5935430
118604,2011-09-16,19:03:00,"29, 104",traill county,102,white,male,vehicular,390902: Exceeded speed limit,citation,White,north dakota,"Traill County, North Dakota",8077,88,1.0895134
74973,2011-01-28,16:46:00,"200, 88",dunn county,11,black,male,vehicular,390902: Exceeded speed limit,citation,African American,north dakota,"Dunn County, North Dakota",4195,458,10.9177592
57595,2010-10-15,10:46:00,"29, 127",grand forks county,11,white,male,vehicular,390902: Exceeded speed limit,citation,White,north dakota,"Grand Forks County, North Dakota",68979,1789,2.5935430
149505,2012-02-21,19:17:00,"85, 174",mckenzie county,11,white,male,vehicular,390902: Exceeded speed limit,citation,White,north dakota,"McKenzie County, North Dakota",9615,1709,17.7743110


In [13]:
## What is the distribution of stops across each factor (ie county, violation, sex, etc)?

nd_data %>%
    count(raw_Race) %>%
    mutate(prop_of_stops = n/sum(n)) %>%
    arrange(desc(prop_of_stops))

## Native American stops seem to be slightly underrepresented at the state level, 
## based on fact that Native Americans make up ~5% of the total ND population. Will pull in census data later.

nd_data %>%
    count(county) %>%
    mutate(prop_of_stops = n/sum(n)) %>%
    arrange(desc(prop_of_stops))

nd_data %>%
    count(subject_sex) %>%
    mutate(prop_of_stops = n/sum(n)) %>%
    arrange(desc(prop_of_stops))

nd_data %>%
    count(type) %>%
    mutate(prop_of_stops = n/sum(n)) %>%
    arrange(desc(prop_of_stops))

nd_data %>%
    count(violation) %>%
    mutate(prop_of_stops = n/sum(n)) %>%
    arrange(desc(prop_of_stops))

nd_data %>%
    count(outcome) %>%
    mutate(prop_of_stops = n/sum(n)) %>%
    arrange(desc(prop_of_stops))

raw_Race,n,prop_of_stops
<chr>,<int>,<dbl>
White,291682,0.8835314359
Native American,13555,0.0410593338
African American,9562,0.0289641719
Hispanic,8713,0.0263924733
Other,4756,0.0144063587
Asian,1691,0.0051221935
NA,173,0.0005240328


county,n,prop_of_stops
<chr>,<int>,<dbl>
cass county,40211,1.218028e-01
ward county,32371,9.805472e-02
grand forks county,31219,9.456520e-02
williams county,20328,6.157537e-02
ramsey county,17096,5.178535e-02
stark county,16373,4.959531e-02
morton county,14808,4.485479e-02
stutsman county,14450,4.377037e-02
burleigh county,14176,4.294040e-02


subject_sex,n,prop_of_stops
<chr>,<int>,<dbl>
male,242904,0.7357784159
female,87137,0.2639459368
NA,91,0.0002756473


type,n,prop_of_stops
<chr>,<int>,<dbl>
vehicular,327573,0.992248555
NA,2559,0.007751445


violation,n,prop_of_stops
<chr>,<int>,<dbl>
390902: Exceeded speed limit,171436,0.519295312
3921414: No seat belt,22670,0.068669502
390902: Exceeded speed limit|3921414: No seat belt,12436,0.037669781
3904371: Failed to register motor vehicle upon gainful employment,11686,0.035397962
391044: Disregarded stop sign,11486,0.034792144
3921394: Vehicle having tinted windshield,7330,0.022203240
3921463: Commercial Motor Vehicle Violations,4612,0.013970170
390801: Drove or in actual physical control of a motor vehicle while under the influence of alcohol or drugs and/or with AC of .08 or greater and/or,4576,0.013861122
3909011: Care required,3862,0.011698351


outcome,n,prop_of_stops
<chr>,<int>,<dbl>
citation,330132,1


In [6]:
nd_na_stops_data <-
    nd_data %>%
    filter(raw_Race == "Native American")

In [7]:
## What is the distribution of Native American stops across relevant factors (ie county, sex, type, violation, etc)

nd_na_stops_data %>%
    count(county) %>%
    mutate(prop_of_stops = n/sum(n)) %>%
    arrange(desc(prop_of_stops))

nd_na_stops_data %>%
    count(subject_sex) %>%
    mutate(prop_of_stops = n/sum(n)) %>%
    arrange(desc(prop_of_stops))

nd_na_stops_data %>%
    count(type) %>%
    mutate(prop_of_stops = n/sum(n)) %>%
    arrange(desc(prop_of_stops))

nd_na_stops_data %>%
    count(violation) %>%
    mutate(prop_of_stops = n/sum(n)) %>%
    arrange(desc(prop_of_stops))

nd_na_stops_data %>%
    count(outcome) %>%
    mutate(prop_of_stops = n/sum(n)) %>%
    arrange(desc(prop_of_stops))

county,n,prop_of_stops
<chr>,<int>,<dbl>
rolette county,2733,0.2016230173
ramsey county,2305,0.1700479528
ward county,1375,0.1014385835
morton county,1127,0.0831427518
mclean county,755,0.0556990041
grand forks county,687,0.0506824050
burleigh county,582,0.0429361859
bottineau county,488,0.0360014755
cass county,364,0.0268535596


subject_sex,n,prop_of_stops
<chr>,<int>,<dbl>
male,7997,0.5899668
female,5558,0.4100332


type,n,prop_of_stops
<chr>,<int>,<dbl>
vehicular,13297,0.98096643
NA,258,0.01903357


violation,n,prop_of_stops
<chr>,<int>,<dbl>
390902: Exceeded speed limit,5350,0.394688307
3921414: No seat belt,804,0.059313906
3904371: Failed to register motor vehicle upon gainful employment,519,0.038288454
390642: Drove while license suspended(4 or more offenses),454,0.033493176
390902: Exceeded speed limit|3921414: No seat belt,452,0.033345629
3921394: Vehicle having tinted windshield,450,0.033198082
390601: Drove without\expired operators license,235,0.017336776
390820: Driving without Liability Insurance,227,0.016746588
390642: Drove while license suspended(4 or more offenses)|390820: Driving without Liability Insurance,208,0.015344891


outcome,n,prop_of_stops
<chr>,<int>,<dbl>
citation,13555,1


#### Glancing at disproportionate stops with census data

In [16]:
nd_data %>% head()

raw_row_number,date,time,location,county,subject_age,subject_race,subject_sex,type,violation,outcome,raw_Race,state,geography,total_pop,total_na,na_pop_prop
<chr>,<date>,<drtn>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
91444,2011-05-04,09:58:00,"94, 291",barnes county,101,white,male,vehicular,390902: Exceeded speed limit,citation,White,north dakota,"Barnes County, North Dakota",11097,104,0.9371902
138130,2011-12-28,11:41:00,gateway at 55th st,grand forks county,101,white,male,vehicular,391022: Failed to yield at intersection,citation,White,north dakota,"Grand Forks County, North Dakota",68979,1789,2.5935430
118604,2011-09-16,19:03:00,"29, 104",traill county,102,white,male,vehicular,390902: Exceeded speed limit,citation,White,north dakota,"Traill County, North Dakota",8077,88,1.0895134
74973,2011-01-28,16:46:00,"200, 88",dunn county,11,black,male,vehicular,390902: Exceeded speed limit,citation,African American,north dakota,"Dunn County, North Dakota",4195,458,10.9177592
57595,2010-10-15,10:46:00,"29, 127",grand forks county,11,white,male,vehicular,390902: Exceeded speed limit,citation,White,north dakota,"Grand Forks County, North Dakota",68979,1789,2.5935430
149505,2012-02-21,19:17:00,"85, 174",mckenzie county,11,white,male,vehicular,390902: Exceeded speed limit,citation,White,north dakota,"McKenzie County, North Dakota",9615,1709,17.7743110


In [8]:
## Sorted by proportion of NA stops - NA proportion in the population (ie the disparity)

nd_data %>%
    group_by(geography, na_pop_prop, raw_Race) %>%
    summarize(
        total_num_stops = n()
    ) %>%
    mutate(prop_of_stops = (total_num_stops/sum(total_num_stops)) * 100) %>%
    filter(raw_Race == "Native American") %>%
    mutate(disprop_stops = prop_of_stops - na_pop_prop) %>%
    arrange(desc(disprop_stops))

geography,na_pop_prop,raw_Race,total_num_stops,prop_of_stops,disprop_stops
<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>
"Towner County, North Dakota",2.1360070,Native American,286,23.0088496,20.87284258
"Pierce County, North Dakota",1.8657565,Native American,361,14.9111937,13.04543718
"Bottineau County, North Dakota",2.4721134,Native American,488,10.5650574,8.09294402
"McHenry County, North Dakota",0.6532577,Native American,238,6.8371158,6.18385808
"Nelson County, North Dakota",1.3123360,Native American,261,6.2906725,4.97833649
"Morton County, North Dakota",3.6329136,Native American,1127,7.6107509,3.97783737
"Ramsey County, North Dakota",9.8132457,Native American,2305,13.4826860,3.66944029
"Ward County, North Dakota",1.3360694,Native American,1375,4.2476291,2.91155960
"Sheridan County, North Dakota",0.4487659,Native American,7,3.0303030,2.58153714


In [18]:
## Sorted by NA % of the Population

nd_data %>%
    group_by(geography, na_pop_prop, raw_Race) %>%
    summarize(
        total_num_stops = n()
    ) %>%
    mutate(prop_of_stops = (total_num_stops/sum(total_num_stops)) * 100) %>%
    filter(raw_Race == "Native American") %>%
    mutate(disprop_stops = prop_of_stops - na_pop_prop) %>%
    arrange(desc(na_pop_prop))

geography,na_pop_prop,raw_Race,total_num_stops,prop_of_stops,disprop_stops
<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>
"Sioux County, North Dakota",81.8036530,Native American,2,9.5238095,-72.27984344
"Rolette County, North Dakota",77.3210098,Native American,2733,65.0559391,-12.26507073
"Benson County, North Dakota",54.9749779,Native American,166,6.5225933,-48.45238460
"Mountrail County, North Dakota",28.3799849,Native American,103,2.0000000,-26.37998487
"McKenzie County, North Dakota",17.7743110,Native American,132,1.2199630,-16.55434794
"Dunn County, North Dakota",10.9177592,Native American,170,3.5617012,-7.35605800
"Ramsey County, North Dakota",9.8132457,Native American,2305,13.4826860,3.66944029
"McLean County, North Dakota",7.1844249,Native American,755,7.5841286,0.39970365
"Oliver County, North Dakota",4.2880704,Native American,24,1.8348624,-2.45320798


In [19]:
nd_data %>%
    group_by(geography, na_pop_prop, raw_Race) %>%
    summarize(
        total_num_stops = n()
    ) %>%
    mutate(prop_of_stops = (total_num_stops/sum(total_num_stops)) * 100) %>%
    filter(raw_Race == "Native American") %>%
    mutate(disprop_stops = prop_of_stops - na_pop_prop) %>%
    arrange(desc(total_num_stops))

geography,na_pop_prop,raw_Race,total_num_stops,prop_of_stops,disprop_stops
<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>
"Rolette County, North Dakota",77.3210098,Native American,2733,65.0559391,-12.26507073
"Ramsey County, North Dakota",9.8132457,Native American,2305,13.4826860,3.66944029
"Ward County, North Dakota",1.3360694,Native American,1375,4.2476291,2.91155960
"Morton County, North Dakota",3.6329136,Native American,1127,7.6107509,3.97783737
"McLean County, North Dakota",7.1844249,Native American,755,7.5841286,0.39970365
"Grand Forks County, North Dakota",2.5935430,Native American,687,2.2005830,-0.39295998
"Burleigh County, North Dakota",3.7915283,Native American,582,4.1055305,0.31400219
"Bottineau County, North Dakota",2.4721134,Native American,488,10.5650574,8.09294402
"Cass County, North Dakota",1.1680000,Native American,364,0.9052249,-0.26277506


In [23]:
nd_data %>%
    group_by(geography, na_pop_prop, raw_Race) %>%
    summarize(
        total_num_stops = n()
    ) %>%
    mutate(prop_of_stops = (total_num_stops/sum(total_num_stops)) * 100) %>%
    filter(raw_Race == "Native American") %>%
    mutate(
        disprop_stops = prop_of_stops - na_pop_prop,
        STATEFP = "38"
    ) %>%
    separate(geography, into = c("county", "state"), sep = ", ", remove = FALSE) %>%
    write_csv("./data/nd_data_prop.csv")

## Washington Analysis

In [ ]:
#wa_data %>%
#    head()

In [10]:
?separate()

separate {tidyr},R Documentation
data,A data frame.
col,Column name or position. This is passed to tidyselect::vars_pull(). This argument is passed by expression and supports quasiquotation (you can unquote column names or column positions).
into,Names of new variables to create as character vector. Use NA to omit the variable in the output.
sep,"Separator between columns. If character, is interpreted as a regular expression. The default value is a regular expression that matches any sequence of non-alphanumeric values. If numeric, interpreted as positions to split at. Positive values start at 1 at the far-left of the string; negative value start at -1 at the far-right of the string. The length of sep should be one less than into."
remove,"If TRUE, remove input column from output data frame."
convert,"If TRUE, will run type.convert() with as.is = TRUE on new columns. This is useful if the component columns are integer, numeric or logical."
extra,"If sep is a character vector, this controls what happens when there are too many pieces. There are three valid options: ""warn"" (the default): emit a warning and drop extra values. ""drop"": drop any extra values without a warning. ""merge"": only splits at most length(into) times"
fill,"If sep is a character vector, this controls what happens when there are not enough pieces. There are three valid options: ""warn"" (the default): emit a warning and fill from the right ""right"": fill with missing values on the right ""left"": fill with missing values on the left"
...,Additional arguments passed on to methods.
